In [1]:
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import pandas as pd
import re as regex
import numpy as np
import scipy
import copy
import re
import nltk

In [2]:
master_data = pd.read_csv("multilabels.csv")

In [3]:
master_data

,label,comment
0,none,@jbromley29 Science is only good when it says ...
1,none,Every day I think maybe it will be the day I f...
2,none,*sigh* oh Colin 😍 #MKR
3,none,"Just read the phrase ""feminist agenda"" and sno..."
4,none,It's insane they keep bringing people back. Wh...
5,none,"Definitely read, forward, read again: http://t..."
6,none,@berkeley_eagle #MKR this shit show has more ...
7,none,"""Sexism can work both ways"" = I can stop for t..."
8,none,@twothug4u educate yourself on oppression. htt...
9,none,Wishing a horde of rabid bats on @Trizzzy13 an...


In [4]:
features = np.array(master_data.ix[:, 1])

/Users/williamscott/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [5]:
labels = np.array(master_data.ix[:, 0])

/Users/williamscott/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [6]:
labels

array(['none', 'none', 'none', ..., 'PersonalAttack', 'PersonalAttack',
       'PersonalAttack'], dtype=object)

In [7]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

# import re
# text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

In [8]:
# test = [["I removed ftp://github.com/ZeerakW/hatespeech/blob/master/NAACL_SRW_2016.csv the following:NEWLINE_TOKENNEWLINE_TOKENAll names of early Polish rulers are ficticious and therefore this index naming Oda von Haldensleben and her husband Dagome records for the first time rulers of the Polanen tribe. Therefore it is indicated as being the first document of the later developing land named Poland.NEWLINE_TOKENNEWLINE_TOKENThis is quite a comment. All names are fictitious? It deserves at least some backing."],
#                  ["Therefore it is indicated as being the first document of the later developing land named Poland.NEWLINE_TOKENNEWLINE_TOKENThis is quite a comment. All names are fictitious? It deserves at least some backing."]]

In [9]:
# test = re.sub(r'^httasdfps?:\/\/.*[\r\n]*', '', test, flags=re.MULTILINE)
test = features
p_data = []

for k in test:
#     print(k)
    p_data.append([re.sub(r"(http|ftp|https)\S+", "", str(k))])
    

In [10]:
# p_data

In [11]:
test = p_data

In [12]:
def lowercase(arr):
    return np.char.lower(arr)

In [13]:
def removeStopWords(arr):
    stopWords = set(stopwords.words('english'))
    
    filteredWords = []
    for a in arr:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            if w not in stopWords:
                wordsFiltered = wordsFiltered + " " + w
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [14]:
def removeSymbols(arr):
    symbols = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
    for i in range(len(symbols)):
        arr = np.char.replace(arr, symbols[i], '')
    
    return np.char.replace(arr, "  ", " ")

In [15]:
def stemming(word):
    porter_stemmer=PorterStemmer()
    filteredWords = []
    for a in word:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            wordsFiltered = wordsFiltered + " " + porter_stemmer.stem(w)
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [16]:
def removeImplicitNoise(arr):
    return np.char.replace (arr, 'NEWLINE_TOKEN', '')

In [17]:
def processClean(data):
     
    a = removeImplicitNoise(data)

    a = lowercase(a)

    a = removeStopWords(a)

    a = removeSymbols(a)
    
    a = stemming(a)

    return a

a = processClean(test)

# print(a)

In [52]:
list_l = []

for i in a:
    list_l.append(str(i[0]))
# print(list_l)

In [47]:
# for i in a:
#     print(i)

In [53]:
# list_l

In [54]:
def make_grams(data, n = 3):
    
    grammed_data = []
    
    for i in data:
#         print("\nT:",i)

        k = copy.deepcopy(i)

        for r in range(2,n+1):
            sixgrams = ngrams(i.split(), r)

            for grams in sixgrams:
                g = ""
                for p in grams:
                    g = g+p

                k = k+" "+g
#         print("\nK:",k)
        grammed_data.append(k)
    
    return grammed_data

grammed_data = make_grams(list_l)

RuntimeError: generator raised StopIteration

In [ ]:
# grammed_data

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, test_size=0.33, random_state=42)

In [ ]:
#Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(feature_vector.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, test_size=0.33, random_state=42)

In [ ]:
gnb = MultinomialNB()
gnb.fit(X_train, y_train)

In [ ]:
score = gnb.score(X_test, y_test)
y_pred = gnb.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print("score:",score)

In [ ]:
gnb.score(X_train, y_train)

In [ ]:
# TF-IDF
vectorizer = TfidfVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(vector.shape)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')